In [2]:
# conda install -c rapidsai -c nvidia -c conda-forge dask-cuda cudatoolkit=11.6
# nvidia-smi
from sklearn.model_selection import train_test_split, KFold
import dask.dataframe as dd
import dask.multiprocessing
import dask
import numpy as np
from dask import delayed
import dask_image.imread
import dask_image.ndfilters
import dask_image.ndmeasure
import cv2
from glob import glob
import json
import dask.array as da
import time
import xgboost as xgb
import pandas as pd
import os
from dask import compute
from sklearn.metrics import f1_score
# f1_score(y_true, y_pred, average=[‘micro’, ‘macro’, ‘samples’,’weighted’ 중 하나 선택])
import tensorflow as tf

@delayed
def padding(data, array_len, col_len):
    pad = np.zeros((array_len, col_len))
    length = min(array_len, len(data))
    
    pad[:length] = data[:length]
    return pad

@delayed
def img_resize(img):
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)/255
    return img

path = os.getcwd()
csv_features = ['내부 온도 1 평균', '내부 온도 1 최고', '내부 온도 1 최저', '내부 습도 1 평균', '내부 습도 1 최고', 
                '내부 습도 1 최저', '내부 이슬점 평균', '내부 이슬점 최고', '내부 이슬점 최저']

# from dask.distributed import Client, LocalCluster
# # client.close()
# cluster = LocalCluster()
# client = Client(cluster)
# print(cluster,client)
# client.close()
# http://localhost:8787/status

# from dask_cuda import LocalCUDACluster
# from dask.distributed import Client
# client.close()
# # http://127.0.0.1:34497/status
# cluster = LocalCUDACluster()
# client = Client(cluster)
# print(cluster,client)

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2022-01-27 22:23:25.135291: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-27 22:23:25.163020: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-01-27 22:23:25.163151: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/gob/miniconda3/envs/pro1/lib/python3.7/site-packages/cv2/../../lib64:
2022-01-27 22:23:25.163191: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dy

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5797616123380717352
 xla_global_id: -1]

ould like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
#making_label_set

crop = {'1':'딸기','2':'토마토','3':'파프리카','4':'오이','5':'고추','6':'시설포도'}
disease = {'1':{'a1':'딸기잿빛곰팡이병','a2':'딸기흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '2':{'a5':'토마토흰가루병','a6':'토마토잿빛곰팡이병','b2':'열과','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '3':{'a9':'파프리카흰가루병','a10':'파프리카잘록병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '4':{'a3':'오이노균병','a4':'오이흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '5':{'a7':'고추탄저병','a8':'고추흰가루병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '6':{'a11':'시설포도탄저병','a12':'시설포도노균병','b4':'일소피해','b5':'축과병'}}
risk = {'0':'정상','1':'초기','2':'중기','3':'말기'}


#ensemble_labler
label_description = {}
for key, value in disease.items():
    label_description[f'{key}_00_0'] = f'{crop[key]}_정상'
    for disease_code in value:
        for risk_code in risk:
            label = f'{key}_{disease_code}_{risk_code}'
            label_description[label] = f'{crop[key]}_{disease[key][disease_code]}_{risk[risk_code]}'

global ensemble_label_encoder
ensemble_label_encoder = {key:idx for idx, key in enumerate(label_description)}
ensemble_label_decoder = {val:key for key, val in ensemble_label_encoder.items()}

#crop_labler
crop_label_description = {}
for key, value in disease.items():
    crop_label_description[f'{key}'] = f'{crop[key]}'
    
global crop_label_encoder
crop_label_encoder = {key:idx for idx, key in enumerate(crop_label_description)}
crop_label_decoder = {val:key for key, val in crop_label_encoder.items()}


#disease_labler
disease_label_description = {}
for key, value in disease.items():
    disease_label_description[f'00'] = "정상"
    for disease_code,value in value.items():
        label = f'{disease_code}'
        disease_label_description[label] = f'{value}'

global disease_label_encoder
disease_label_encoder = {key:idx for idx, key in enumerate(disease_label_description)}
disease_label_decoder = {val:key for key, val in disease_label_encoder.items()}


#risk_labler
risk_label_description = {}
for key, value in risk.items():
    label = f'{key}'
    risk_label_description[label] = f'{value}'

global risk_label_encoder
risk_label_encoder = {key:idx for idx, key in enumerate(risk_label_description)}
risk_label_decoder = {val:key for key, val in risk_label_encoder.items()}





In [5]:
crop_label_description

{'1': '딸기', '2': '토마토', '3': '파프리카', '4': '오이', '5': '고추', '6': '시설포도'}

In [6]:
risk_label_description

{'0': '정상', '1': '초기', '2': '중기', '3': '말기'}

In [7]:
disease_label_description

{'00': '정상',
 'a1': '딸기잿빛곰팡이병',
 'a2': '딸기흰가루병',
 'b1': '냉해피해',
 'b6': '다량원소결핍 (N)',
 'b7': '다량원소결핍 (P)',
 'b8': '다량원소결핍 (K)',
 'a5': '토마토흰가루병',
 'a6': '토마토잿빛곰팡이병',
 'b2': '열과',
 'b3': '칼슘결핍',
 'a9': '파프리카흰가루병',
 'a10': '파프리카잘록병',
 'a3': '오이노균병',
 'a4': '오이흰가루병',
 'a7': '고추탄저병',
 'a8': '고추흰가루병',
 'a11': '시설포도탄저병',
 'a12': '시설포도노균병',
 'b4': '일소피해',
 'b5': '축과병'}

In [8]:
@delayed
def label_encoding(label):
    global ensemble_label_encoder
    encoded_label = ensemble_label_encoder[label]
    return encoded_label

@delayed
def label_encoding_crop(label):
    global crop_label_encoder
    encoded_label = crop_label_encoder[label]
    return encoded_label

@delayed
def label_encoding_disease(label):
    global disease_label_encoder
    encoded_label = disease_label_encoder[label]
    return encoded_label

@delayed
def label_encoding_risk(label):
    global risk_label_encoder
    encoded_label = risk_label_encoder[label]
    return encoded_label

def getlable(jsonpath,type):
    if type == "ensemble":
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)

        crop = json_file['annotations']['crop']
        disease = json_file['annotations']['disease']
        risk = json_file['annotations']['risk']
        label = f'{crop}_{disease}_{risk}'
        return label

    elif type == "crop":
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)
        crop = json_file['annotations']['crop']
        label = f'{crop}'
        return label

    elif type == "dc":
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)
        disease = json_file['annotations']['disease']
        label = f'{disease}'
        return label

    elif type == "risk":  
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)
        risk = json_file['annotations']['risk']
        label = f'{risk}'
        return label

    
def get_label_list(labelpath_list):
    labelarr = np.array([])
    labelarr_crop = np.array([])
    labelarr_dc = np.array([])
    labelarr_risk = np.array([])


    # labelarr = da.array([])
    for ind,json_path in enumerate(labelpath_list):
        # label = label_encoding(getlable(json_path))
        # label = da.array(np.array(label_encoding(getlable(json_path))))
        label = np.array(label_encoding(getlable(json_path,"ensemble")))
        labelarr = np.append(labelarr,label)

        label_crop = np.array(label_encoding_crop(getlable(json_path,"crop")))
        labelarr_crop = np.append(labelarr_crop,label_crop)

        label_dc = np.array(label_encoding_disease(getlable(json_path,"dc")))
        labelarr_dc = np.append(labelarr_dc,label_dc)

        label_risk = np.array(label_encoding_risk(getlable(json_path,"risk")))
        labelarr_risk = np.append(labelarr_risk,label_risk)


    return labelarr, labelarr_crop, labelarr_dc, labelarr_risk

In [9]:
@delayed
def get_all_image(imgpath_list,image_size):
    imgarr = np.empty((0,image_size,image_size,3), float)
    for ind,img_path in enumerate(imgpath_list):
        imgarr = delayed(np.append)(imgarr,img, axis = 0)
    return imgarr

In [14]:
%%time
@delayed
def imageresize(img):
    # img = dask.delayed(cv2.resize)(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    # img = dask.delayed(img.astype(np.float32)/255)  ##픽셀값을 0~1사이로 정규화
    # # img = np.transpose(img, (2,0,1))
    # return dask.delayed(img.reshape)(-1,224,224,3)
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)/255  ##픽셀값을 0~1사이로 정규화
    # img = np.transpose(img, (2,0,1))
    return da.array(img)
#making_img_set
row_img = dask_image.imread.imread(os.path.join(path,"data","train","*","*.jpg"))
train = [imageresize(img) for img in row_img]
train_x = da.array(dask.compute(*train))

CPU times: user 45.4 s, sys: 3.31 s, total: 48.7 s
Wall time: 13.4 s


In [11]:
%%time
label_list = sorted(glob(os.path.join(path,"data","train","*","*.json")))
y_train,y_train_crop,y_train_dc,y_train_risk, = get_label_list(label_list)
# y_train = y_train.rechunk(5767)
results = dask.compute(*y_train)
label = np.array(results)
# print(label.shape)
results_crop = dask.compute(*y_train_crop)
label_crop = np.array(results_crop)
#
results_dc = dask.compute(*y_train_dc)
label_disease = np.array(results_dc)
#
results_risk = dask.compute(*y_train_risk)
label_risk = np.array(results_risk)
label.shape,label_crop.shape,label_disease.shape,label_risk.shape

CPU times: user 3.89 s, sys: 430 ms, total: 4.32 s
Wall time: 5.86 s


((5767,), (5767,), (5767,), (5767,))

In [12]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  
])
model.compile()

2022-01-27 22:24:09.916905: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-01-27 22:24:09.917476: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [13]:
%%time
batch = 100
data_length = len(train_x)
loop_num = int(np.floor(data_length/batch))

train_x_feature = dask.array.zeros((data_length,model(train_x[:1]).numpy().shape[1]))


count = 0
for i in range(loop_num):
    print(i)
    if i != loop_num-1:
        start_p = count*batch
        end_p = (count+1)*batch
        train_x_feature[start_p:end_p] = model(train_x[start_p:end_p]).numpy()
        
    else:
        start_p = count*batch
        end_p = (count+1)*batch
        train_x_feature[start_p:] = model(train_x[start_p:]).numpy()
            
    count += 1



0


2022-01-27 22:24:12.411711: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 60211200 exceeds 10% of free system memory.
2022-01-27 22:24:12.420311: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 160563200 exceeds 10% of free system memory.
2022-01-27 22:24:12.443230: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 160563200 exceeds 10% of free system memory.
2022-01-27 22:24:12.470464: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 160563200 exceeds 10% of free system memory.
2022-01-27 22:24:12.502208: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 160563200 exceeds 10% of free system memory.


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
CPU times: user 12min 48s, sys: 5min 33s, total: 18min 21s
Wall time: 1min 26s


In [15]:
#making_csv_set
row_csv = dd.read_csv(os.path.join(path,"data","train","*","*.csv"),dtype={'외부 누적일사 평균': 'object','내부 이슬점 최고': 'object',
       '내부 이슬점 최저': 'object',
       '내부 이슬점 평균': 'object'})
partitions = row_csv.to_delayed()
# datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features].values,290,9) for part in partitions]
datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features][part != "-"].dropna().values,290,9) for part in partitions[:]]
X_train = da.array(dask.compute(*datas))


In [16]:
X_csv, X_csv_test,X_predimg, X_predimg_test, y, y_test,y_crop, y_crop_test,y_risk, y_risk_test,y_disease, y_disease_test, = train_test_split(X_train,train_x_feature, label,label_crop,label_risk,label_disease, test_size=0.2, shuffle= True,stratify=label)

트레이닝셋 준비

In [17]:
params = {"tree_method": "gpu_hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
        #   "single_precision_histogram" : True,
          }
models_csv = []
models_predimg = []

for j in [y,y_crop,y_risk,y_disease]:
  for i in range(len(csv_features)):
    start = time.time()
    _ = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X_csv[:,:,i],j)
    models_csv.append(_)
    print(time.time()-start)

for j in [y,y_crop,y_risk,y_disease]:
    start = time.time()
    _ = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X_predimg,j)
    models_predimg.append(_)
    print(time.time()-start)

/home/gob/miniconda3/envs/pro1/lib/python3.7/site-packages/xgboost/core.py:502: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  format(", ".join(args_msg)), FutureWarning
/home/gob/miniconda3/envs/pro1/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


10.503957986831665
10.575292348861694
10.568800210952759
10.854691505432129
10.677682638168335
10.896791696548462
9.990189790725708
10.202274322509766
10.237291097640991
3.7663826942443848
3.8355729579925537
3.653353691101074
3.7650692462921143
3.726477861404419
3.737304449081421
3.830886125564575
3.6188080310821533
3.6424524784088135
3.0370283126831055
3.2497007846832275
3.2129693031311035
3.538980484008789
3.4550325870513916
3.574117422103882
3.136270761489868
3.1745126247406006
3.025207042694092
4.717905759811401
4.636017084121704
4.627770662307739
4.901976108551025
4.782014846801758
4.954428195953369
4.44329833984375
4.409198760986328
4.469293594360352
55.105998516082764
20.22113800048828
20.36137866973877
31.949662923812866


In [27]:
processed_x = da.zeros((len(X_predimg),len(models_predimg)+len(models_csv)))
processed_x_val = da.zeros((len(X_predimg_test),len(models_predimg)+len(models_csv)))


for k,j in enumerate([y,y_crop,y_risk,y_disease]):
  for i in range(len(csv_features)):
    iter = i+k*len(csv_features)
    processed_x[:,iter] = models_csv[iter].predict(X_csv[:,:,i])
    processed_x_val[:,iter] = models_csv[iter].predict(X_csv_test[:,:,i])

for k,j in enumerate([y,y_crop,y_risk,y_disease]):
    processed_x[:,k+len(models_csv)] = models_predimg[k].predict(X_predimg)
    processed_x_val[:,k+len(models_csv)] = models_predimg[k].predict(X_predimg_test)
   
    
   

In [29]:
processed_x_val

dask.array<setitem, shape=(1154, 40), dtype=float64, chunksize=(1154, 40), chunktype=numpy.ndarray>

In [30]:
params = {"tree_method": "gpu_hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
        #   "single_precision_histogram" : True,
          }

start = time.time()
XGB_final = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(processed_x,y)
print(time.time()-start)

/home/gob/miniconda3/envs/pro1/lib/python3.7/site-packages/xgboost/core.py:502: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  format(", ".join(args_msg)), FutureWarning
/home/gob/miniconda3/envs/pro1/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


2.566976308822632


In [31]:
y_pred = XGB_final.predict(processed_x_val)

In [32]:
from sklearn import metrics

# Print the confusion matrix
print(metrics.confusion_matrix(y_test, y_pred))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_test, y_pred, digits=3))

[[162   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0  28   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0  38   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0 235   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0   0  26   1   1   0   1   0   1   0   0   0   0   0   1   0
    0   0   0   0   0   0   0]
 [  0   0   1   0  13   2   0   0   3   0   2   1   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   1   0   4   1   0   0   1   0   0   0   0   0   0   0   1   0
    0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0  19  11   0   3   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   1  22   1   4   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   1   0  27   1   2   0  

In [33]:
answer = np.array([label_description[ensemble_label_decoder[int(val)]] for val in y_test])
predss = np.array([label_description[ensemble_label_decoder[int(val)]] for val in y_pred])

new_crosstab = pd.crosstab(answer, predss, rownames=['answer'], colnames=['preds'])
new_crosstab

preds,고추_고추탄저병_중기,고추_다량원소결핍 (K)_초기,고추_다량원소결핍 (N)_초기,고추_다량원소결핍 (P)_초기,고추_정상,딸기_정상,시설포도_시설포도노균병_중기,시설포도_시설포도노균병_초기,시설포도_시설포도탄저병_중기,시설포도_시설포도탄저병_초기,...,토마토_정상,토마토_토마토흰가루병_중기,파프리카_다량원소결핍 (K)_초기,파프리카_다량원소결핍 (N)_초기,파프리카_다량원소결핍 (P)_초기,파프리카_정상,파프리카_칼슘결핍_초기,파프리카_파프리카흰가루병_말기,파프리카_파프리카흰가루병_중기,파프리카_파프리카흰가루병_초기
answer,,,,,,,,,,,,,,,,,,,,,
고추_고추탄저병_중기,20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
고추_다량원소결핍 (K)_초기,0,21,4,0,0,0,0,0,0,0,...,0,0,1,5,0,0,0,0,0,0
고추_다량원소결핍 (N)_초기,0,5,18,0,0,0,0,0,0,0,...,0,0,0,7,0,0,0,0,0,0
고추_다량원소결핍 (P)_초기,0,0,0,32,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
고추_정상,0,0,2,0,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
딸기_정상,0,0,0,0,0,162,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
시설포도_시설포도노균병_중기,0,0,0,0,0,0,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
시설포도_시설포도노균병_초기,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,1
시설포도_시설포도탄저병_중기,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred,average='macro')

0.7835976864406419